In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

# Load Data

In [ ]:
sp_merge <- readRDS(file = '/home/jupyter/Organized_Gating_Analysis/Seurat_Objects/sp_merge_gating_celltype_updated.rds')

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'gating_celltype')
non_naive <- subset(sp_merge, idents = c('CD4 Naive','CD8 Naive','CD8 Unk','CD4 Unk'), invert=TRUE)

In [ ]:
non_naive$pure_celltype <- non_naive$gating_celltype

In [ ]:
cd4na <- readRDS(file = '../Figure3/cd4_na_labeled.rds')

In [ ]:
cd8na <- readRDS(file = '../Figure4/trim_cd8_na.rds')

In [ ]:
all_merge <- merge(cd4na, c(cd8na, non_naive))

In [ ]:
rm(sp_merge)
rm(cd4na)
rm(cd8na)
rm(non_naive)

# Normalize Data

In [ ]:
all_merge <- NormalizeData(all_merge, assay = 'clean_adt')

In [ ]:
avg_exp <- AverageExpression(all_merge, assay = 'clean_adt', features = rownames(all_merge[['clean_adt']]))

In [ ]:
head(avg_exp$clean_adt)

# Pheatmap

In [ ]:
library(pheatmap)

In [ ]:
mat <- as.data.frame(avg_exp$clean_adt)

In [ ]:
drop_adts <- c('CD16','HLA.DR')
mat2 <- mat[!(rownames(mat) %in% drop_adts),]
length(rownames(mat2))
length(rownames(mat))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
adt_pheatmap <- pheatmap(mat2, cluster_rows = TRUE, cluster_cols = TRUE, scale = 'row', cellwidth = 25, cellheight = 12, show_rownames = TRUE, border_color = NA,
                         # annotation_row = cell_state_genes, annotation_colors = anno_colors,
                         color = colorRampPalette(c('#67a9cf','white','#d6604d'))(100))

In [ ]:
save_pheatmap_pdf <- function(x, filename, width=10, height=10) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}
save_pheatmap_pdf(adt_pheatmap, "plots/ADT_heatmap.pdf")

# CD8 Pops only

In [ ]:
sp_merge <- SetIdent(sp_merge, value = 'gating_celltype')
cd8_mem <- subset(sp_merge, idents = c('CD8 CM','CD8 EM1','CD8 EM2','CD8 TEMRA'))

In [ ]:
cd8_mem$pure_celltype <- cd8_mem$gating_celltype

In [ ]:
cd8na <- readRDS(file = '../Figure4/trim_cd8_na.rds')

In [ ]:
cd8_merge <- merge(cd8na, cd8_mem)

In [ ]:
rm(sp_merge)
rm(cd8_mem)
rm(cd8na)

## Normalize Data

In [ ]:
cd8_merge <- NormalizeData(cd8_merge, assay = 'clean_adt')

In [ ]:
avg_exp <- AverageExpression(cd8_merge, assay = 'clean_adt', features = rownames(cd8_merge[['clean_adt']]))

In [ ]:
head(avg_exp$clean_adt)

## Pheatmap - ExtFig6A

In [ ]:
library(pheatmap)

In [ ]:
mat <- as.data.frame(avg_exp$clean_adt)

In [ ]:
drop_adts <- c('CD16','HLA.DR','CD8a','CD4')
mat2 <- mat[!(rownames(mat) %in% drop_adts),]
length(rownames(mat2))
length(rownames(mat))

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
adt_pheatmap <- pheatmap(mat2, cluster_rows = TRUE, cluster_cols = TRUE, scale = 'row', cellwidth = 25, cellheight = 12, show_rownames = TRUE, border_color = NA,
                         # annotation_row = cell_state_genes, annotation_colors = anno_colors,
                         color = colorRampPalette(c('#67a9cf','white','#d6604d'))(100))

In [ ]:
save_pheatmap_pdf <- function(x, filename, width=10, height=10) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}
save_pheatmap_pdf(adt_pheatmap, "plots/ADT_heatmap_CD8_only.pdf")